# PyTwoWay example

In [1]:
# Add PyTwoWay to system path, do not run this
# import sys
# sys.path.append('../../..')

In [2]:
# Import the PyTwoWay package 
# (Make sure you have installed it using pip install pytwoway)
import pytwoway as tw
import bipartitepandas as bpd

## Simulate some data

The package contains functions to simulate data. We use this here to keep things simple. If you have your own data, you can import it. Load it as a pandas dataframe and use it as an input.

As you can see, we will need the following required columns in our data:

 - `i`: worker identifier
 - `j`: firm identifier
 - `y`: compensation
 - `t`: time

In [3]:
# For the example, we simulate data
sim_data = bpd.SimBipartite().sim_network()
display(sim_data)

,i,t,k,alpha,psi,spell,freq,j,move,y
0,0,1,6,0.430727,0.348756,1,5,111,False,1.538809
1,0,2,6,0.430727,0.348756,1,5,111,False,0.821696
2,0,3,6,0.430727,0.348756,1,5,111,False,0.888712
3,0,4,6,0.430727,0.348756,1,5,111,False,0.048582
4,0,5,6,0.430727,0.348756,1,5,111,False,0.720645
...,...,...,...,...,...,...,...,...,...,...
49995,9999,1,2,-0.430727,-0.604585,1,1,30,False,-2.295800
49996,9999,2,0,-0.430727,-1.335178,2,1,9,True,-2.129870
49997,9999,3,4,-0.430727,-0.114185,3,1,65,True,-2.167106
49998,9999,4,5,-0.430727,0.114185,4,2,92,True,0.058889


## Create a TwoWay object using your data

In [4]:
# We need to specify a column dictionary to make sure columns are named correctly
# You can also manually update column names yourself
col_name_dict = {
    'i': 'i',    # Specify the column name for the worker identifier 
    'j': 'j',    # Specify the column name for the firm identifier 
    'y': 'y',  # Specify the column name for the compensation variable
    't': 't'   # Specify the column name for the time variable
}

# Create the TwoWay object that will do all the heavy lifting
tw_net = tw.TwoWay(data=sim_data, formatting='long', col_dict=col_name_dict)

## Clean data

In [5]:
## Optional Parameters ##
clean_params = {
    'connectedness': 'connected', # When computing largest connected set of firms:
        # If 'connected', keep observations in the largest connected set of firms;
        # If 'biconnected', keep observations in the largest biconnected set of firms;
        # If None, keep all observations
    'i_t_how': 'max', # How to handle worker-year duplicates
        # If 'max', keep max paying job;
        # If 'sum', sum over duplicate worker-firm-year observations,
            # then take the highest paying worker-firm sum;
        # If 'mean', average over duplicate worker-firm-year observations,
            # then take the highest paying worker-firm average.
        # Note that if multiple time and/or firm columns are included
            # (as in event study format), then data is converted to long,
            # cleaned, then reconverted to its original format
    'copy': False # If False, avoid copy
}

In [6]:
tw_net.prep_data(
    collapsed=True, # If True, run estimators on collapsed data
    user_clean=clean_params,
    he=False # If True, compute largest biconnected set of firms for heteroskedastic correction
)

## Now we can run the CRE estimator

In [7]:
## Optional Parameters ##
cre_params = {
    'ncore': 1, # Number of cores to use
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'ndp': 50, # Number of draw to use in approximation for leverages
    'out': 'res_cre.json', # Outputfile where results are saved
    'posterior': False, # If True, compute posterior variance
    'wo_btw': False # If True, sets between variation to 0, pure RE
}
cluster_params = {
    'measures': bpd.measures.cdfs(
        cdf_resolution=10, # How many values to use to approximate the cdfs
        measure='quantile_all' # How to compute the cdfs (
                               # 'quantile_all' to get quantiles from entire set of data,
                                    # then have firm-level values between 0 and 1;
                               # 'quantile_firm_small' to get quantiles at the firm-level
                                    # and have values be compensations if small data;
                               # 'quantile_firm_large' to get quantiles at the firm-level
                                    # and have values be compensations if large data,
                                    # note that this is up to 50 times slower than
                                    # 'quantile_firm_small' and should only be used
                                    # if the dataset is too large to copy into a dictionary
    ),
    'grouping': bpd.grouping.kmeans( # Read more at
                                  # https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html
        n_clusters=10,
        init='k-means++',
        n_init=500,
        max_iter=300,
        tol=0.0001,
        precompute_distances='deprecated',
        verbose=0,
        random_state=None,
        copy_x=True,
        n_jobs='deprecated',
        algorithm='auto'
    ),
    'stayers_movers': None, # If None, clusters on entire dataset;
                         # If 'stayers', clusters on only stayers
                         # If 'movers', clusters on only movers
    't': None, # If None, clusters on entire dataset
            # If int, gives period in data to consider (only valid for non-collapsed data)
    'weighted': True, # If True, weight firm clusters by firm size
                        # (if a weight column is included, firm weight is computed using this column;
                        # otherwise, each observation has weight 1)
    'dropna': False # If True, drop observations where firms aren't clustered;
                 # If False, keep all observations
}

In [8]:
# Cluster
tw_net.cluster(**cluster_params)
# Estimate the cre model
tw_net.fit_cre(user_cre=cre_params)

## We can also run the FE estimator

In [9]:
# First, compute largest biconnected set of firms for heteroskedastic correction
tw_net.prep_data(
    collapsed=True, # If True, run estimators on collapsed data
    user_clean=clean_params,
    he=True # If True, compute largest biconnected set of firms for heteroskedastic correction
)

In [10]:
## Optional Parameters ##
fe_params = {
    'ncore': 1, # Number of cores to use
    'batch': 1, # Batch size to send in parallel
    'ndraw_pii': 50, # Number of draws to use in approximation for leverages
    'levfile': '', # File to load precomputed leverages
    'ndraw_tr': 5, # Number of draws to use in approximation for traces
    'he': True, # If True, compute heteroskedastic correction
    'out': 'res_fe.json', # Outputfile where results are saved
    'statsonly': False, # If True, return only basic statistics
    'Q': 'cov(alpha, psi)' # Which Q matrix to consider. Options include 'cov(alpha, psi)' and 'cov(psi_t, psi_{t+1})'
}

In [ ]:
# Estimate the fixed effect decomposition
tw_net.fit_fe(user_fe=fe_params)

## Finally, we can investigate the results

In [12]:
display(tw_net.summary_cre())
display(tw_net.summary_fe())

{'var_y': 1.988764409495749,
 'var_bw': 0.5904416718602248,
 'cov_bw': 0.1835031454491588,
 'var_tot': 0.5891609808891859,
 'cov_tot': 0.18465501303507245}

{'var_y': 1.9561198864792677,
 'var_fe': 0.5986416982327974,
 'cov_fe': 0.17803898603112833,
 'var_ho': 0.5916841556340435,
 'cov_ho': 0.18054370240332968,
 'var_he': 0.588987090687847,
 'cov_he': 0.18105849433400442}